# **Importing approprite packages**

In [ ]:
%tensorflow_version 1.X

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.X`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.


In [ ]:
import tensorflow
print(tensorflow.__version__)
print(tensorflow.keras.__version__)

1.15.2
2.2.4-tf


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import matplotlib.pyplot as plt
import keras
import numpy as np
import os
from keras import backend as K
import cv2
from keras.preprocessing.image import ImageDataGenerator
import pickle
from keras.callbacks import ModelCheckpoint
from skimage import img_as_ubyte

from keras.models import *
from keras.layers import *
from keras.optimizers import *

seed_value = 5421
np.random.seed(seed_value)

startDir = '/content/drive/My Drive/Colab Notebooks/swri_research/data/'
weights_dir = startDir

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Using TensorFlow backend.


# **Designing Data Generator**

In [ ]:
from scipy.interpolate import interp1d
from skimage.measure import label, regionprops
import csv
def imageLoaderNew(files, batch_size):
  L = len(files)
  k = 0
  num = np.int(L/batch_size) 
  #this line is just to make the generator infinite, keras needs that    
  while True:
    k = k % num
    f_batch = files[batch_size*k:batch_size*(k+1)]
    X = np.zeros((batch_size,90,90,225),dtype=float)
    X_b=np.zeros((90,90,225*batch_size))
    for r in range(batch_size):
      y = fits.open(f_batch[r])
      yy = fits.open(f_batch[r][:-5]+'.dt.fits')
      imgs = np.transpose(y[0].data)
      imgs=np.clip(imgs,-1000,1000)
      imgs=0.5*(1+imgs/1000)
      frame=yy[0].data/5760
      frame=frame.astype('int')
      mx=int(np.max(frame)+1)
      if r==0:
        ind_q=frame
        ind_i=np.arange(mx)
      else:
        ind_q=np.append(ind_q,225*r+frame)
        ind_i=np.append(ind_i,225*r+np.arange(mx))

      for i in range(len(frame)):
        X[r,:,:,frame[i]] = imgs[:,:,i]
      for i in range(mx,225):
        X[r,:,:,i] = imgs[:,:,len(frame)-1]
      
      X_b[:,:,r*225:(r+1)*225]=X[r,:,:,:]
    
    for i in range(90):
      for j in range(90):
        f_q = X_b[i,j,ind_q]
        f = interp1d(ind_q, f_q)
        X_b[i,j,ind_i]=f(ind_i)
   
    for l in range(batch_size):
      X[l,:,:,:]=X_b[:,:,l*225:(l+1)*225]

    im = (X==0)
    for i in range(batch_size):
      for j in range(225):
        label_image = label(im[i,:,:,j])
        mn=np.min(label_image)
        mx=np.max(label_image)+1
        for r in range(mn,mx):
          bw=(label_image == r)*im[i,:,:,j]
          if bw[0,0]==1 or bw[0,89]==1 or bw[89,0]==1 or bw[89,89]==1:
            X[i,:,:,j]=0.5*bw+X[i,:,:,j]

    Y = np.zeros(batch_size)
    for i in range(batch_size):
      fl = open('/content/drive/My Drive/Colab Notebooks/swri_research/data/BARD_v2/BARD_v2_data_file_path_labels_pass_f.csv', 'r')
      reader = csv.reader(fl)
      for row in reader:
        if row[35] == f_batch[i][58:]:
          Y[i] = int(row[36])

    k = k+1

    yield(X,Y)
    yield (1-X,Y) 
    yield(np.flip(X,2),Y)
    yield(1-np.flip(X,2),Y)

# **Creating Training, Validation combinations for Model Ensemble**

In [ ]:
import pathlib
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import glob,os
import pickle

s=(800,90,90,200)
data_learn=np.zeros(s)

f_test_y = []
f_test_n = []

f_name_y=[]
dir_path_y='/content/drive/My Drive/Colab Notebooks/swri_research/data/BARD_v2/Emergence/'
for root, dirs, files in os.walk(dir_path_y+'Training/'): 
    for file in files:  
        if file.endswith('dt.fits'):
          fh = root+file
          fd = fh[:-8]+'.fits'
          f = pathlib.Path(fd)
          if f.exists() and int(fd[-9:-7])<11:
            f_name_y.append(fd)


for root, dirs, files in os.walk(dir_path_y+'Validation/'): 
    for file in files:  
        if file.endswith('dt.fits'):
          fh = root+file
          fd = fh[:-8]+'.fits'
          f = pathlib.Path(fd)
          if f.exists() and int(fd[-9:-7])<11:
            f_name_y.append(fd)




for root, dirs, files in os.walk(dir_path_y+'Test/'): 
    for file in files:  
        if file.endswith('dt.fits'):
          fh = root+file
          fd = fh[:-8]+'.fits'
          f = pathlib.Path(fd)
          if f.exists():
            if f.exists() and fd[-9:-7]=='10':
              f_name_y.append(fd)
            if fd[-9:-7]=='11' or fd[-9:-7]=='12':
              f_test_y.append(fd)







f_name_n=[]
dir_path_n='/content/drive/My Drive/Colab Notebooks/swri_research/data/BARD_v2/AR_Non_Emergence/'
for root, dirs, files in os.walk(dir_path_n+'Training/'): 
    for file in files:  
        if file.endswith('dt.fits'):
          fh = root+file
          fd = fh[:-8]+'.fits'
          f = pathlib.Path(fd)
          if f.exists() and int(fd[-9:-7])<11:
            f_name_n.append(fd)


for root, dirs, files in os.walk(dir_path_n+'Validation/'): 
    for file in files:  
        if file.endswith('dt.fits'):
          fh = root+file
          fd = fh[:-8]+'.fits'
          f = pathlib.Path(fd)
          if f.exists() and int(fd[-9:-7])<11:
            f_name_n.append(fd)



for root, dirs, files in os.walk(dir_path_n+'Test/'): 
    for file in files:  
        if file.endswith('dt.fits'):
          fh = root+file
          fd = fh[:-8]+'.fits'
          f = pathlib.Path(fd)
          if f.exists():
            if f.exists() and fd[-9:-7]=='10':
              f_name_n.append(fd)
            if fd[-9:-7]=='11' or fd[-9:-7]=='12':
              f_test_n.append(fd)

In [ ]:
f_yn=[]
f_yn_test = []
for i in range(len(f_name_y)):
  f_yn.append(f_name_y[i])

for i in range(len(f_name_n)):
  f_yn.append(f_name_n[i])

for i in range(len(f_test_y)):
  f_yn_test.append(f_test_y[i])

for i in range(len(f_test_n)):
  f_yn_test.append(f_test_n[i])


num=[]
for i in range(len(f_yn)):
    num.append(f_yn[i][-13:-5])

num_test=[]
for i in range(len(f_yn_test)):
    num_test.append(f_yn_test[i][-13:-5])

arg=np.argsort(num)

arg_test = np.argsort(num_test)

f_final=[]

for i in range(len(arg)):
    f_final.append(f_yn[arg[i]])

f_final_test = []

for i in range(len(arg_test)):
    f_final_test.append(f_yn_test[arg_test[i]])

val_months=['05','06']
files_train=[]
files_val=[]
for i in range(len(f_final)):
  if f_final[i][-9:-7]==val_months[0] or f_final[i][-9:-7]==val_months[1]:
    files_val.append(f_final[i])
  else:
    files_train.append(f_final[i])


print(files_val)
print(len(files_val))
print(files_train)
print(len(files_train))
print(len(f_final))
print(f_final_test)
print(len(f_final_test))


['/content/drive/My Drive/Colab Notebooks/swri_research/data/BARD_v2/AR_Non_Emergence/Training/BARD0000_19960509.fits', '/content/drive/My Drive/Colab Notebooks/swri_research/data/BARD_v2/Emergence/Training/BARD0002_19960627.fits', '/content/drive/My Drive/Colab Notebooks/swri_research/data/BARD_v2/AR_Non_Emergence/Training/BARD0003_19960630.fits', '/content/drive/My Drive/Colab Notebooks/swri_research/data/BARD_v2/Emergence/Training/BARD0047_19970508.fits', '/content/drive/My Drive/Colab Notebooks/swri_research/data/BARD_v2/AR_Non_Emergence/Training/BARD0046_19970508.fits', '/content/drive/My Drive/Colab Notebooks/swri_research/data/BARD_v2/AR_Non_Emergence/Training/BARD0049_19970517.fits', '/content/drive/My Drive/Colab Notebooks/swri_research/data/BARD_v2/Emergence/Training/BARD0050_19970520.fits', '/content/drive/My Drive/Colab Notebooks/swri_research/data/BARD_v2/Emergence/Training/BARD0051_19970522.fits', '/content/drive/My Drive/Colab Notebooks/swri_research/data/BARD_v2/AR_Non_

In [ ]:
fileList_train = files_train[:int(10*np.floor(0.1*len(files_train)))]
print(len(fileList_train))
fileList_valid = files_val[:int(10*np.floor(0.1*len(files_val)))]
print(len(fileList_valid))

1580
440


# **Designing the CNN and Training**

In [ ]:
from keras.backend import sigmoid
def swish(x, beta = 1):
    return (x * sigmoid(beta * x))

from keras.utils.generic_utils import get_custom_objects
from keras.layers import Activation
get_custom_objects().update({'swish': Activation(swish)})

In [ ]:
# #####Model with BatchNorm#####
# #(x,y,ch)
# #(90,90,225)
model = Sequential() 
model.add(Conv2D(32, (3, 3), input_shape = (90,90,225),padding='same')) 
model.add(Activation('swish'))
model.add(BatchNormalization()) 
model.add(Conv2D(32, (3, 3),padding='same')) 
model.add(Activation('swish')) 
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size =(2, 2), strides=(2,2))) 

#(45,45,32)
model.add(Conv2D(64, (3, 3))) 
model.add(Activation('swish'))
model.add(BatchNormalization()) 
model.add(Conv2D(64, (3, 3))) 
model.add(Activation('swish')) 
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size =(2, 2), strides=(2,2))) 

#(20,20,64)
model.add(Conv2D(128, (3, 3))) 
model.add(Activation('swish'))
model.add(BatchNormalization()) 
model.add(Conv2D(128, (3, 3))) 
model.add(Activation('swish')) 
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size =(2, 2), strides=(2,2))) 



#(8,8,128)
model.add(Conv2D(256, (3, 3))) 
model.add(Activation('swish'))
model.add(BatchNormalization()) 
model.add(Conv2D(256, (3, 3))) 
model.add(Activation('swish')) 
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size =(2, 2), strides=(2,2))) 


#(2,2,256)
model.add(Conv2D(512, (2, 2))) 
model.add(Activation('swish'))


#(1,1,512)  
model.add(Flatten()) 
model.add(Dense(512)) 
model.add(Activation('swish')) 
model.add(Dropout(0.5,seed=seed_value)) 
model.add(Dense(512)) 
model.add(Activation('swish')) 
model.add(Dropout(0.5,seed=seed_value))
model.add(Dense(1)) 
model.add(Activation('sigmoid'))

In [ ]:
sgd = SGD(lr=0.0001, momentum=0.9, nesterov=True)
model.compile(loss ='binary_crossentropy',  optimizer = sgd, metrics =['accuracy']) 

In [ ]:
batch_size=10
from keras.callbacks import ModelCheckpoint, EarlyStopping
checkpoint = ModelCheckpoint("/content/drive/My Drive/Colab Notebooks/swri_research/code/flux_emrg_weights_newData_vgg_56_pass_f_v4.h5", monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=True, mode='auto', period=1)
early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=5, verbose=1, mode='auto')
history=model.fit(imageLoaderNew(fileList_train, batch_size), steps_per_epoch = 4*len(fileList_train)/batch_size, epochs = 20,callbacks=[checkpoint,early_stopping], validation_data=imageLoaderNew(fileList_valid, batch_size), validation_steps = 4*len(fileList_valid)/batch_size)


Epoch 1/20
  4/632 [..............................] - ETA: 48:31 - loss: 0.7941 - accuracy: 0.4750  

/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:718: UserWarning: An input could not be retrieved. It could be because a worker has died.We do not have any information on the lost sample.
  UserWarning)


632/632 [==============================] - 2428s 4s/step - loss: 0.6342 - accuracy: 0.6456 - val_loss: 0.6116 - val_accuracy: 0.6477

Epoch 00001: val_accuracy improved from -inf to 0.64773, saving model to /content/drive/My Drive/Colab Notebooks/swri_research/code/flux_emrg_weights_newData_vgg_56_pass_f_v4.h5
Epoch 2/20
632/632 [==============================] - 1102s 2s/step - loss: 0.5217 - accuracy: 0.7150 - val_loss: 0.5332 - val_accuracy: 0.6960

Epoch 00002: val_accuracy improved from 0.64773 to 0.69602, saving model to /content/drive/My Drive/Colab Notebooks/swri_research/code/flux_emrg_weights_newData_vgg_56_pass_f_v4.h5
Epoch 3/20
632/632 [==============================] - 1109s 2s/step - loss: 0.4408 - accuracy: 0.7826 - val_loss: 0.3647 - val_accuracy: 0.7426

Epoch 00003: val_accuracy improved from 0.69602 to 0.74261, saving model to /content/drive/My Drive/Colab Notebooks/swri_research/code/flux_emrg_weights_newData_vgg_56_pass_f_v4.h5
Epoch 4/20
632/632 [================

# **Estimating Uncertainty of Emergence Probability on Test Set using Model Ensemble**

In [ ]:
###for uncertainty of emergence probability##
from scipy.interpolate import interp1d
from skimage.measure import label, regionprops
from astropy.io import fits
import csv
new_row=[]
with open('/content/drive/My Drive/Colab Notebooks/swri_research/data/BARD_v2/BARD_v2_file_test_path_label_f.csv', 'r') as f:
  reader = csv.reader(f)
  for row in reader:
    if row[0]!='Path':
      path = '/content/drive/My Drive/Colab Notebooks/swri_research/data' + row[0] 
      y = fits.open(path)
      yy = fits.open(path[:-5]+'.dt.fits')
      imgs = np.transpose(y[0].data)
      imgs=np.clip(imgs,-1000,1000)
      imgs=0.5*(1+imgs/1000)
      frame=yy[0].data/5760
      frame=frame.astype('int')
      lst = frame.tolist()
      miss = []
      mn=int(np.min(frame))
      mx=int(np.max(frame)+1)
      for i in range(mn,mx):
        if i not in lst:
          miss.append(i)

      X = np.zeros((1,90,90,225),dtype=float)
      for i in range(len(frame)):
        X[0,:,:,frame[i]] = imgs[:,:,i]
      for i in range(mx,225):
        X[0,:,:,i] = imgs[:,:,len(frame)-1]
      ind_q=frame
      ind_i=np.arange(mx)
      for i in range(90):
        for j in range(90):
          f_q = X[0,i,j,ind_q]
          f = interp1d(ind_q, f_q, kind = 'linear')
          X[0,i,j,ind_i]=f(ind_i)

      im = (X==0)
      for j in range(225):
        label_image = label(im[0,:,:,j])
        mn=np.min(label_image)
        mx=np.max(label_image)+1
        for r in range(mn,mx):
          bw=(label_image == r)*im[0,:,:,j]
          if bw[0,0]==1 or bw[0,89]==1 or bw[89,0]==1 or bw[89,89]==1:
            X[0,:,:,j]=0.5*bw+X[0,:,:,j]

    
      model.load_weights('/content/drive/My Drive/Colab Notebooks/swri_research/code/flux_emrg_weights_newData_vgg_12_pass_f_v3.h5')
      row[5] = model.predict(X)[0][0]
      model.load_weights('/content/drive/My Drive/Colab Notebooks/swri_research/code/flux_emrg_weights_newData_vgg_34_pass_f_v3.h5')
      row[6] = model.predict(X)[0][0]
      model.load_weights('/content/drive/My Drive/Colab Notebooks/swri_research/code/flux_emrg_weights_newData_vgg_56_pass_f_v3.h5')
      row[7] = model.predict(X)[0][0]
      model.load_weights('/content/drive/My Drive/Colab Notebooks/swri_research/code/flux_emrg_weights_newData_vgg_78_pass_f_v3.h5')  
      row[8] = model.predict(X)[0][0]
      model.load_weights('/content/drive/My Drive/Colab Notebooks/swri_research/code/flux_emrg_weights_newData_vgg_9X_pass_f_v3.h5')  
      row[9] = model.predict(X)[0][0]
      model.load_weights('/content/drive/My Drive/Colab Notebooks/swri_research/code/flux_emrg_weights_newData_vgg_12_pass_f_v4.h5')
      row[10] = model.predict(X)[0][0]
      model.load_weights('/content/drive/My Drive/Colab Notebooks/swri_research/code/flux_emrg_weights_newData_vgg_34_pass_f_v4.h5')
      row[11] = model.predict(X)[0][0]
      model.load_weights('/content/drive/My Drive/Colab Notebooks/swri_research/code/flux_emrg_weights_newData_vgg_56_pass_f_v4.h5')
      row[12] = model.predict(X)[0][0]
      model.load_weights('/content/drive/My Drive/Colab Notebooks/swri_research/code/flux_emrg_weights_newData_vgg_78_pass_f_v4.h5')  
      row[13] = model.predict(X)[0][0]
      model.load_weights('/content/drive/My Drive/Colab Notebooks/swri_research/code/flux_emrg_weights_newData_vgg_9X_pass_f_v4.h5')  
      row[14] = model.predict(X)[0][0]
      print(row)
      new_row.append(row)
    else:
      new_row.append(row)

with open('/content/drive/My Drive/Colab Notebooks/swri_research/data/BARD_v2/BARD_v2_file_test_path_label_f.csv', 'w') as f:
    # Overwrite the old file with the modified rows
    writer = csv.writer(f)
    writer.writerows(new_row)


['/BARD_v2/Emergence/Test/BARD0013_19961111.fits', '1', 'bipolar emergence 53', '', '1', 0.7802088, 0.6490103, 0.8751311, 0.67352015, 0.52056116, 0.76242656, 0.72549593, 0.3888586, 0.75903314, 0.55157185]
['/BARD_v2/AR_Non_Emergence/Test/BARD0014_19961113.fits', '1', 'tail of an emergence', '', '0', 0.4152993, 0.5911135, 0.97344524, 0.6939124, 0.58526564, 0.8307033, 0.9382685, 0.79472005, 0.27957603, 0.75038266]
['/BARD_v2/Emergence/Test/BARD0016_19961115.fits', '1', 'bipolar emergence 40', '', '0', 0.4401176, 0.5694816, 0.79427695, 0.6097, 0.47996926, 0.84767216, 0.97712505, 0.5098274, 0.39497727, 0.45528007]
['/BARD_v2/AR_Non_Emergence/Test/BARD0015_19961117.fits', '1', 'bipolar emergence 35', '', '1', 0.93687123, 0.9414163, 0.99777645, 0.88791716, 0.8974958, 0.9641716, 0.9996019, 0.99961215, 0.9994147, 0.9896694]
['/BARD_v2/Emergence/Test/BARD0018_19961123.fits', '1', 'tail of an emergence', '', '1', 0.999049, 0.86364853, 0.992879, 0.90982705, 0.9918944, 0.99885476, 0.9999989, 0.99